In [91]:
!pip install transformers[sentencepiece]==4.28.0 datasets sacrebleu evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [92]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value = user_secrets.get_secret("HF_KEY")
login(secret_value)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [93]:
import transformers

print(transformers.__version__)

4.29.2


In [94]:
from datasets import load_dataset

#wmt16_train = load_dataset("ethansimrm/wmt16_biomed", use_auth_token=True) #We won't need to use this for now
wmt16_test = load_dataset("ethansimrm/wmt16_biomed_test", use_auth_token=True)
wmt16_gold = load_dataset("ethansimrm/wmt16_biomed_gold", use_auth_token=True)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Let's aim low for now. Given a (small) pre-trained large language model - Marian or T5-Small, etc, we wish to evaluate baseline performance on the WMT16 Biomedical Test Set. 
- We have converted the test set and the gold-standard answers to CSV files and uploaded both to the HuggingFace Hub.
- Intuitively, we must extract source sentences from the test set and target sentences from the gold set, but only where there is a direct correspondence between them.
- We know for sure that "passage/0" (the scientific paper titles) - are one sentence long in both languages, so there is a direct correspondence - we cannot guarantee the same for the rest of the sentences.



Additional Information:
- We could simply truncate the gold-standard answers to the length of the source sentences, but we only require a quick-and-dirty evaluation today.
- Furthermore, the test set contains French abstracts which contain information which cannot be inferred from the English source; we cannot assume a 1-1 faithful translation.

In [95]:
#The relevant columns for the abstract titles are:
source_sentences = wmt16_test["test"]["passage/0/sentence/0/text"]
target_sentences = wmt16_gold["train"]["passage/0/sentence/0/text"]

In [96]:
from datasets import load_metric
metric = load_metric("sacrebleu")

In [97]:
#This used a pretrained model from an earlier tutorial - it's quite bad. We need to train our base models
#on at least some out of domain EN-FR data, and upload them to the Hub before evaluating on the test set.
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, T5Model, pipeline
tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("ethansimrm/test_t5_small_example_kaggle3") 
translator = pipeline("translation", model=model, tokenizer=tokenizer, use_auth_token = True)
#for s in source_sentences:
    #source = "translate English to French: " + s
    #if(source[len(source) - 1] == '.'):
        #source = source[:-1]
    #print(source)
    #result = translator(source)
    #print(result)

Let's choose a smaller (ish?) but still somewhat relevant OOD corpus - the UN Parallel Corpus, which is often incorporated in training data for WMT tasks.

In [105]:
training_data = load_dataset("un_pc", "en-fr") 

Generating train split:   0%|          | 0/30340652 [00:00<?, ? examples/s]

Dataset un_pc downloaded and prepared to /root/.cache/huggingface/datasets/un_pc/en-fr/1.0.0/1360070a820db42f7427f5a98416dd3a1c956ae069b994bf2ec0b83ae16dcaee. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoTokenizer
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
source_lang = "en"
target_lang = "fr"
prefix = "translate English to French: "
def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]] 
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs